# Google Colab でバイオインフォマティクス（導入編）
だいたい2000年くらいにいろいろな生物のゲノムが決定されて、その出てきたDNA配列を処理するのにバイオインフォマティクスは発展してきました。
ここではそういうDNAやアミノ酸の配列をどうやって処理するかについて、少しだけ紹介します。
DNAは物質ですし、その配列を決めるのは化学や生物の領域なのですが、読み取ったデータは文字列なので情報学的に扱えます。

## 相補鎖配列を得る
DNAは、AとT、CとGがペアになっていて、2本の鎖が互いに水素結合をして2本鎖になっています。
その相方の配列のことを相補鎖といいます。

今回、元の塩基配列は`CTGCTACATCCAGAACTGCCCCCTGGGAGG`です。
`seq`という変数に代入。

In [2]:
seq="CTGCTACATCCAGAACTGCCCCCTGGGAGG"

これを、相方になるように文字列を変換します。

In [11]:
rev=seq.translate(str.maketrans('GATC', 'CTAG'))
print(rev)

GACGATGTAGGTCTTGACGGGGGACCCTCC


これでできあがり**ではありません**。
元の配列と一緒に表示してみると

In [5]:
print("5'-"+seq+"-3'")
print("3'-"+rev+"-5'")

5'-CTGCTACATCCAGAACTGCCCCCTGGGAGG-3'
3'-GACGATGTAGGTCTTGACGGGGGACCCTCC-5'


ちゃんとあっているじゃないかー、という感じもしますが、DNAの配列は特に注記しないと5'を左に書く習わしがあるので、左右を反転しなければなりません。（※　注記は3'、5'を明示するという意味だと思ってください）
なので

In [8]:
comp = rev[::-1]
print(comp)

CCTCCCAGGGGGCAGTTCTGGATGTAGCAG


ということで、相補鎖の配列は`CCTCCCAGGGGGCAGTTCTGGATGTAGCAG`になります。

このようなバイオインフォマティクス用のプログラムを集めたBioPythonというのもあります。Google　Colabにはインストールされていないのでまずはインストール

In [13]:
!pip install biopython

     |████████████████████████████████| 2.3MB 4.2MB/s 


BioPythonで相補鎖配列を取得する例です（がんばればもっと短く書けます）

In [18]:
from Bio.Seq import Seq
seq="CTGCTACATCCAGAACTGCCCCCTGGGAGG"    # さっき定義したが、コピペしやすいようにもう一度 書いておく
seq_f = Seq(seq)                                 # BioPython用に（オブジェクトとして）格納
seq_r = seq_f.reverse_complement()             #　相補鎖に変換
print(str(seq_r))                                  #　文字列として出力

CCTCCCAGGGGGCAGTTCTGGATGTAGCAG


## 翻訳：DNA→アミノ酸（タンパク質）配列への変換
DNAに生命情報が書かれている、などとよく言われますが、DNA→（転写）→mRNA→（翻訳）→（アミノ酸→）タンパク質の処理が生物内で行われて、タンパク質が実際の生物活動を支えています。
翻訳ですが、DNA3塩基が1アミノ酸に対応しています。アミノ酸は20種類あって、各々、1文字表記があるので、AUG　→　M（AUGの配列がメチオニンに対応）などと64通り（4×4×4）書いてもいいのですが、めんどくさいので、これはBioPythonで。

シンプルバージョンをまずは

In [21]:
seq_aa = seq_f.translate()
print(str(seq_aa))

LLHPELPPGR


元の配列は`CTGCTACATCCAGAACTGCCCCCTGGGAGG`ですが、


*  CTG　CTA　CAT　...
*  TGC　TAC　ATC　...
*  GCT ACA TCC ...

そして相補鎖の

* CCT CCC AGG ...
* CTC CAG GGG ...
* TCC CAG GGG ... 

の6パターンが考えられます。なので、本来は次のように6フレーム分考えます。（実際はスタートのMから終止コドンの*までの長さが一番長いものを採用したりします）




In [29]:
seq_f_aa1 = seq_f[1:].translate()
seq_f_aa2 = seq_f[2:].translate()
seq_f_aa3 = seq_f[3:].translate()
seq_r_aa1 = seq_r[1:].translate()            # ... = seq_f.reverse_complement()[1:].translate() というように書いてもよい
seq_r_aa2 = seq_r[2:].translate()
seq_r_aa3 = seq_r[3:].translate()

print(str(seq_f_aa1))
print(str(seq_f_aa2))
print(str(seq_f_aa3))
print(str(seq_r_aa1))
print(str(seq_r_aa2))
print(str(seq_r_aa3))

CYIQNCPLG
ATSRTAPWE
LHPELPPGR
LPGGSSGCS
SQGAVLDVA
PRGQFWM*Q


/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,
